# Single Agent on JudgeBench with Position Swapping


## Imports

In [2]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm

c:\Users\nikob\Desktop\BA\Bachelorarbeit\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [7]:
# Load and prepare a subset of the JudgeBench dataset
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")
df = pd.DataFrame(JudgeBench_Claude)
df_sampled = df.sample(n=100, random_state=42).reset_index(drop=True)
df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [8]:
# Load Azure OpenAI configuration from environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

# Define the model configuration for Azure OpenAI API access
config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,
        "temperature": 0,
        "cache_seed": 42
    }
]

## System Design

In [9]:
# Define the system prompt for the Evaluator-Agent
agent_system_message = f"""
You are an objective Evaluator-Agent.
Your task is to choose the more factually correct response to the question between Response A and Response B.
Give an explanation on your decision using about 200 words.
Always begin your output with: "As an objective Evaluator-Agent I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

In [10]:
# Initialize the ConversableAgents for system setup
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
)

agent = ConversableAgent(
    "Evaluator-Agent", 
    llm_config={"config_list": config_list},
    system_message=agent_system_message,
    human_input_mode="NEVER",
)

## Evaluation

In [11]:
# Define the evaluation function
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    result = initializer.initiate_chat(agent, message=message, max_turns=1)
    result_str = str(result)

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    # If pattern not found, assign "X" to indicate invalid response
    system_decision = match.group(1) if match else "X"

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }

In [ ]:
# Prepare evaluation data
num_rows = 100
df_final_subset = df_final.head(num_rows)

# Evaluate responses in normal position
results_1 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/single_1.csv', index=False)

# Evaluate responses in swapped positions
results_2 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"],  # swap positions
        response_B=row["response_A"],  # swap positions
        label=row["label"],
    )

    # Correct system decision and correctness flag after swapping
    # If system_decision is "X", leave it unchanged
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/single_2.csv', index=False)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

initializer (to Evaluator-Agent):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mole

Progress:   6%|▌         | 6/100 [00:00<00:01, 53.65it/s]

initializer (to Evaluator-Agent):

 
    Question: There are 3 people standing in a line numbered 1 through 3 in a left to right order.
Each person has a set of attributes: Nationality, Hobby, Pet, Music-Genre.
The attributes have the following possible values:
- Nationality: malaysian, japanese, mexican
- Hobby: baking, collecting, hiking
- Pet: snake, guinea-pig, dog
- Music-Genre: funk, pop, electronic
and exactly one person in the line has a given value for an attribute.

Given the following premises about the line of people:
- the person that has a dog is somewhere to the left of the person who is mexican
- the person who listens to pop is not anywhere to the right of the person who listens to electronic
- the person that has a dog is not anywhere to the left of the person who listens to electronic
- the person who listens to electronic is somewhere to the right of the person who likes hiking
- the person who likes hiking does not own a snake
- the person who is malaysian is not a

Progress:  13%|█▎        | 13/100 [00:00<00:01, 58.48it/s]

...

Progress:  92%|█████████▏| 92/100 [00:01<00:00, 65.82it/s]

initializer (to Evaluator-Agent):

 
    Question: The programming contest World Tour Finals is underway, where N players are participating, and half of the competition time has passed.
There are M problems in this contest, and the score A_i of problem i is a multiple of 100 between 500 and 2500, inclusive.
For each i = 1, \ldots, N, you are given a string S_i that indicates which problems player i has already solved.
S_i is a string of length M consisting of o and x, where the j-th character of S_i is o if player i has already solved problem j, and x if they have not yet solved it.
Here, none of the players have solved all the problems yet.
The total score of player i is calculated as the sum of the scores of the problems they have solved, plus a bonus score of i points.
For each i = 1, \ldots, N, answer the following question.

- At least how many of the problems that player i has not yet solved must player i solve to exceed all other players' current total scores?

Note that under t

Progress:  99%|█████████▉| 99/100 [00:01<00:00, 65.86it/s]

initializer (to Evaluator-Agent):

 
    Question: Neural networks:
(A) Are incapable of handling non-linear relationships
(B) Can only handle numerical data
(C) None of the above
(D) Can only be trained with stochastic gradient descent
(E) Are only applicable to visual data
(F) Always require large amounts of data to train
(G) Optimize a convex objective function
(H) Can use a mix of different activation functions
(I) Can only use one type of activation function
(J) Cannot be used for classification tasks
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's analyze each option:

(A) False. Neural networks are actually very good at handling non-linear relationships.
(B) False. Neural networks can handle various types of data, including categorical data.
(C) This might be co

Progress: 100%|██████████| 100/100 [00:01<00:00, 65.51it/s]
